In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [3]:
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden): # this network is the same as the previous one except with an extra hidden layer + dropout
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))

    h2 = tf.nn.dropout(h2, p_keep_hidden)

    return tf.matmul(h2, w_o)

In [4]:
X = tf.placeholder('float', [None, 52])
Y = tf.placeholder('float', [None, 2])

w_h = init_weights([52, 40])
w_h2 = init_weights([40, 30])
w_o = init_weights([30, 2])

p_keep_input = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')

In [5]:
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [7]:
def LeaveOneOut(data1, data2, columnName, useLOO=False):
    grpOutcomes = data1.groupby(columnName).mean().reset_index()
    outcomes = data2['outcome'].values
    x = pd.merge(data2[[columnName, 'outcome']], grpOutcomes,
                 suffixes=('x_', ''),
                 how='left',
                 on=columnName,
                 left_index=True)['outcome']
    if(useLOO):
        x = ((x*x.shape[0])-outcomes)/(x.shape[0]-1)
    return x.fillna(x.mean())

In [8]:
directory = 'data/'
train = pd.read_csv(directory+'act_train.csv')
people = pd.read_csv(directory+'people.csv')

In [9]:
train = pd.merge(train, people,how='left',on='people_id',left_index=True)
train.fillna('-999', inplace=True)

In [10]:
features = train.columns.tolist()
features.remove('activity_id')
features.remove('people_id')
lootrain = pd.DataFrame()
for col in features:
    if(col != 'outcome'):
        print(col)
        lootrain[col] = LeaveOneOut(train, train, col, True).values

date_x
activity_category
char_1_x
char_2_x
char_3_x
char_4_x
char_5_x
char_6_x
char_7_x
char_8_x
char_9_x
char_10_x
char_1_y
group_1
char_2_y
date_y
char_3_y
char_4_y
char_5_y
char_6_y
char_7_y
char_8_y
char_9_y
char_10_y
char_11
char_12
char_13
char_14
char_15
char_16
char_17
char_18
char_19
char_20
char_21
char_22
char_23
char_24
char_25
char_26
char_27
char_28
char_29
char_30
char_31
char_32
char_33
char_34
char_35
char_36
char_37
char_38


In [11]:
y_train = map(lambda x: [1,0] if x==0 else [0,1], train.outcome.as_matrix())
y_train = np.matrix(y_train)
label = train['outcome'].values

In [12]:
from sklearn.metrics import roc_auc_score

In [13]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    
    for i in range(50):

        for start, end in zip(range(0, len(train), 500), range(500, len(train)+1,500)):
            sess.run(train_op, feed_dict={X: lootrain[start:end], Y: y_train[start:end], 
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        predict = sess.run(predict_op, feed_dict={X: lootrain, Y: y_train,
                                                 p_keep_input: 1.0, p_keep_hidden: 1.0})
        print(i, np.mean(label == predict))
        print(i, roc_auc_score(label, predict))
        
        
        #print(i, sess.run(predict_op, feed_dict={X: train, Y: y_train}) )
    


(0, 0.96898590127570727)
(0, 0.96807035546066555)
(1, 0.96740440842837838)
(1, 0.96585039104607062)
(2, 0.9699707503466769)
(2, 0.96988937605445313)
(3, 0.96320651201866303)
(3, 0.96047232072035038)
(4, 0.96644504528530817)
(4, 0.96461399882195098)
(5, 0.96623069042743992)
(5, 0.96427979761604765)
(6, 0.96732795064468025)
(6, 0.96575704834747378)
(7, 0.96823452150853029)
(7, 0.96698295972575288)
(8, 0.96714545319668632)
(8, 0.96549158362212628)
(9, 0.96989884362153211)
(9, 0.96962302673594791)
(10, 0.96968357855195331)
(10, 0.96923727451144226)
(11, 0.96787589809451724)
(11, 0.9664857606163304)
(12, 0.96327022683841146)
(12, 0.96052775367185272)
(13, 0.96874833601921639)
(13, 0.9677151799810283)
(14, 0.97012230059650728)
(14, 0.97000260960933549)
(15, 0.9685116809744363)
(15, 0.96737322196386621)
(16, 0.96874105432553081)
(16, 0.9677386997925862)
(17, 0.96987062705850069)
(17, 0.96967545789666798)
(18, 0.96943873160177696)
(18, 0.96890375899427894)
(19, 0.96966537431773947)
(19, 0.9692